In [53]:
import collections
from itertools import combinations
import time
import hand_evaluator
import concurrent.futures
from multiprocessing import Pool

In [54]:
deck = [i for i in range(0, 52)]
hands = []
for i in range(0, len(deck)):
    for j in range(i + 1, len(deck)):
        hand = (i, j)
        hands.append(hand)

In [55]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
#         print('{}  {:2.2f} ms'.format(method.__name__, (te - ts) * 1000))
        return result
    return timed

In [56]:
# @timeit
def rollouts(hand):
    rem_cards = [i for i in range(0, 52) if i not in hand]
    return combinations(rem_cards, 5)

In [57]:
# @timeit
def possible_opponent_hands(hand, rollout):
    rem_cards = [i for i in range(0, 52) if i not in hand and i not in rollout]
    return combinations(rem_cards, 2)

In [74]:
# @timeit
def compute_win(hero, villan, rollout):
    hero_rank = hand_evaluator.GetRank(rollout[0], rollout[1], rollout[2], 
                                       rollout[3], rollout[4], hero[0], hero[1])
    villan_rank = hand_evaluator.GetRank(rollout[0], rollout[1], rollout[2], 
                                         rollout[3], rollout[4], villan[0], villan[1])
    print("hello")
    if hero_rank > villan_rank:
        return 1
    return 0

In [84]:
def rollout_strength(ins):
    rollout = ins[0]
    hand = ins[1]
    wins = 0
    opponent_hands = 0
    for op_hand in possible_opponent_hands(hand, rollout):
        opponent_hands += 1
        wins += compute_win(hand, op_hand, rollout)
    return wins / opponent_hands

In [105]:
def generate_hand_strength(hand):
    ts = time.time()
    distribution = []
    total = sum(1 for _ in rollouts(hand))
    i = 0
    ins = [(rollout, hand) for rollout in rollouts(hand)]
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(rollout_strength, ini): ini for ini in ins}
#         results = executor.map(rollout_strength, ins, chunksize=1000)
        for future in concurrent.futures.as_completed(futures):
            print(future.result())
#     for result in results:
#         print(result)
        
#     for rollout in rollouts(hand):
#         progress = i / total * 100
#         if time.time() - ts >= 60:
#             print(progress)
#             ts = time.time()
#         distribution.append(rollout_strength(rollout, hand))
#         i += 1
    return distribution

In [106]:
dst = generate_hand_strength((4, 8))
sum(dst) / len(dst)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [108]:
start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ProcessPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
#     results = executor.map(do_something, secs, chunksize=100)
    futures = {executor.submit(do_something, s): s for s in secs}
    for future in concurrent.futures.as_completed(futures):
        print(future.result())

# for result in results:
#     print(result)

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.